In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os

def scrape_amazon_to_json(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        return

    soup = BeautifulSoup(response.content, "html.parser")

    # 1. Get Text Information
    try:
        title = soup.find("span", {"id": "productTitle"}).get_text(strip=True)
    except: title = "N/A"

    try:
        price = soup.find("span", {"class": "a-offscreen"}).get_text(strip=True)
    except: price = "N/A"

    try:
        rating = soup.find("span", {"class": "a-icon-alt"}).get_text(strip=True)
    except: rating = "N/A"

    # 2. Get High-Resolution Image URL
    image_url = "N/A"
    img_tag = soup.find("img", {"id": "landingImage"})
    if img_tag:
        # Amazon stores multiple sizes in this attribute; we pick the largest one
        dynamic_data = img_tag.get("data-a-dynamic-image", "{}")
        image_urls = list(json.loads(dynamic_data).keys())
        if image_urls:
            image_url = image_urls[0]

    # 3. Download the image for your video
    local_image_path = "product_image.jpg"
    if image_url != "N/A":
        img_data = requests.get(image_url).content
        with open(local_image_path, 'wb') as f:
            f.write(img_data)

    # 4. Create the JSON Object
    product_data = {
        "title": title,
        "price": price,
        "rating": rating,
        "image_url": image_url,
        "local_file": local_image_path,
        "source_url": url
    }

    with open('product_info.json', 'w', encoding='utf-8') as f:
        json.dump(product_data, f, indent=4)

    print(f"Success! Data and image saved for: {title[:50]}...")
    return product_data

# Execute
amazon_url = "https://www.amazon.com/Sony-Exclusive-Features-PlayStation%C2%AE5-K-43S20M2/dp/B0DYKBNW89/"
scrape_amazon_to_json(amazon_url)

Success! Data and image saved for: Sony BRAVIA 2 II 43 Inch 4K Ultra HD LED Smart TV ...


{'title': 'Sony BRAVIA 2 II 43 Inch 4K Ultra HD LED Smart TV with Google TV and Exclusive Features for PlayStation®5 (K-43S20M2), 2025 Model',
 'price': 'N/A',
 'rating': '4.4 out of 5 stars.',
 'image_url': 'https://m.media-amazon.com/images/I/71YwNEpGxbL._AC_SX522_.jpg',
 'local_file': 'product_image.jpg',
 'source_url': 'https://www.amazon.com/Sony-Exclusive-Features-PlayStation%C2%AE5-K-43S20M2/dp/B0DYKBNW89/'}

In [ ]:
pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1


In [ ]:
import json
import os
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import ImageClip, AudioFileClip, CompositeVideoClip

def create_high_quality_video(json_path):
    # 1. Load Data
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. Audio Generation
    script = f"Check out the {data['title']}. It's currently {data['price']} with a {data['rating']} rating."
    audio_temp = "speech.mp3"
    gTTS(text=script, lang='en').save(audio_temp)
    audio_clip = AudioFileClip(audio_temp)
    duration = audio_clip.duration

    # 3. Restore the "Better" Frame Layout
    # We create a beautiful canvas with the product image and text
    canvas = Image.new('RGB', (1920, 1080), color=(25, 25, 25))

    # Load and place product image
    prod_img = Image.open(data['local_file']).convert("RGB")
    prod_img.thumbnail((1000, 700)) # Adjust size
    canvas.paste(prod_img, ((1920 - prod_img.size[0]) // 2, 100))

    # Add Text (Using Pillow for stability, but with better styling)
    draw = ImageDraw.Draw(canvas)
    try:
        # These paths usually work on Linux/Colab. Change for Windows/Mac.
        font_main = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf", 45)
        font_price = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf", 90)
    except:
        font_main = font_price = ImageFont.load_default()

    # Draw Title (Wrapped to fit)
    draw.text((960, 820), f"{data['title'][:60]}...", fill="white", font=font_main, anchor="mm")
    # Draw Price in Gold/Yellow
    draw.text((960, 930), f"Price: {data['price']}", fill="#FFD700", font=font_price, anchor="mm")

    frame_path = "styled_frame.png"
    canvas.save(frame_path)

    # 4. Final Video Assembly
    video_clip = ImageClip(frame_path).set_duration(duration)
    final_video = video_clip.set_audio(audio_clip)

    # Export with AAC for guaranteed audio compatibility
    final_video.write_videofile("sony_tv_review.mp4", fps=24, codec="libx264", audio_codec="aac")

    # Cleanup
    audio_clip.close()
    print("Success! High-quality video with audio generated.")

create_high_quality_video('product_info.json')

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Moviepy - Building video sony_tv_review.mp4.
MoviePy - Writing audio in sony_tv_reviewTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video sony_tv_review.mp4



Moviepy - Done !
Moviepy - video ready sony_tv_review.mp4
Success! High-quality video with audio generated.


# New Section